In [78]:
import pickle 

with open('../static/networks/distributeGraph.pkl', 'rb') as f:
    distributeGraph = pickle.load(f)
    print(distributeGraph.number_of_nodes(), distributeGraph.number_of_edges()) # 588150 709997
with open('../static/networks/safeGraph_noUnpopular.pkl', 'rb') as f:
    safeGraph = pickle.load(f)
    print(safeGraph.number_of_nodes(), safeGraph.number_of_edges()) # 55717 915478

588150 709997
48778 621494


In [ ]:
def removeEdgeByTypes(typeNames):
    for edge in safeGraph.edges():
        edgeType = safeGraph.edges[edge]
        for typeName in typeNames:
            if edgeType[typeName] and sum([edgeType['transfer'], edgeType['creator'], edgeType['distributeEth'], edgeType['same_owner'], edgeType['similar_owner']]) == 1:
                safeGraph.remove_edge(edge[0], edge[1])

# a <-- 'creator', 'similar_owner', 'same_owner', 'transfer', 'distributeEth' --> b
# a <-- 'creator' = True, 'similar_owner' = False, 'same_owner' = False, 'transfer' = False, 'distributeEth' = False --> b
# a <-- 'creator' = False, 'similar_owner' = False, 'same_owner' = False, 'transfer' = True, 'distributeEth' = False --> b

removeEdgeByTypes(['transfer', 'creator'])
print(safeGraph.number_of_nodes(), safeGraph.number_of_edges()) # 55717 199791

In [79]:
for edge in safeGraph.edges():
    edgeType = safeGraph.edges[edge]
    if not (edgeType['creator'] and edgeType['same_owner'] and edgeType['transfer']):
        safeGraph.remove_edge(edge[0], edge[1])
    else:
        if edgeType['transfer']:
            records = sorted(list(edgeType['transfer-tx_hash']))
            multisigCount = 0
            popularTokenTransferCount = 0
            for r in records:
                if r.startswith('multisig'):
                    multisigCount += 1
                if r.split('_')[-1] in ('ETH', 'USDC', 'USDT', 'WBTC', 'DAI', 'WETH', 'SHIB', 'HEX'):
                    popularTokenTransferCount += 1
            if multisigCount > 5 or popularTokenTransferCount > 3:
                safeGraph.remove_edge(edge[0], edge[1])

print(safeGraph.number_of_nodes(), safeGraph.number_of_edges())

48778 244


In [80]:
import networkx as nx
allCcs = [set(cc) for cc in sorted(nx.connected_components(safeGraph), key=len, reverse=True)]

for i in range(len(allCcs)):
    if len(allCcs[i]) >= 2:
        print('# Connected Components {} - {} addresses'.format(i + 1, len(allCcs[i])))
        for addr in allCcs[i]:
            print('\t{}'.format(addr))
        print('\n\n\n\n\n')

for addr in safeGraph.nodes():
    if safeGraph.nodes[addr]['similar_owner']:
        print('# Owners connected in distributeGraph:')
        print('\t{}'.format(addr))

# Connected Components 1 - 29 addresses
	0x5969539242892ccf3b2e78437904bd2b0cde684b
	0x998ac90e6286b506f22791a0e01978a0aae30b78
	0xa022ba0a753fa6b5b725e7bc88b178f5a6c707ab
	0x3cd6fff983498df18bad8474554355fafdb63978
	0x5b9962a5dbb9953573f793333b61c2237e5bb5a8
	0x873d8cfd921062ee62b96ed2ea39cd10f321366b
	0xbb390d88be59f9d485617f77cd4d7b8b77bdb6ee
	0x99178b08591085666deb185fcd47c0f0b3d0cb43
	0x718e4c28e4f3f93a38e65969ecd66dc77fdeabd2
	0xeb1063da6f7080eaa0c30356ec6a76cc895ca084
	0xff6746e27f1dd7a21134dd7a6740121424daaff8
	0x08ef2b3b8540eccc1d04a3022c19ea11af29da5a
	0xd6510c2097b93bbd676306c8f59d55579ca56fb1
	0x370222d8bed9635d8f97f74813e256fc856c667a
	0xdd21df577b00302c44a2f801d85af420a3e154d9
	0x53a0035b08da9a27b97bffa7e7be4d917635ae41
	0xb92c4d4b53b9d1bbf0f19fc69812c9f9c5900446
	0x41890bd02fc108db38e8897296f0454758bdb857
	0xdd405d77870f19698ffc3ebb4de18586e02325de
	0xd3438ea85fe7a6819aefa1d14fb439ff8280a216
	0xf501410d751a7c2536dbc9747e42d37ad0434255
	0x07535d4a5e4b39f4ffd47b0fe3dd60db7

In [95]:
# get all evidences between addr1 and addr2

import networkx as nx
from collections.abc import Iterable

def checkEvidence(graph, addr1, addr2): 
    assert addr1 in graph.nodes(), addr1
    assert addr2 in graph.nodes(), addr2
    if (addr1, addr2) in graph.edges:
        edgeInfo = graph.edges[addr1, addr2]
        for k, v in edgeInfo.items():
            print('"{}"'.format(k))
            if isinstance(v, Iterable):
                v = sorted(list(v))
                for element in v:
                    print('\t{}'.format(element))
            else:
                print('\t{}'.format(v))
    else:
        all_shortest_paths = [p for p in nx.all_shortest_paths(graph, source=addr1, target=addr2)]
        for p in all_shortest_paths:
            print(p)

addr1 = '0x6b49e6c79e604ea46e1dc8a936bfad4eb97d59e3'
addr2 = '0x70496a025eeaeb3f13c8153ed6408e0ba54163fb'
graph = safeGraph
checkEvidence(graph, addr1, addr2)

"same_owner"
	True
"creator"
	True
"creator-tx_hash"
	0xabf83c8c2be67a5fd670da5bb23ac1f287f0d95451f8795f266c0fabc2970064
	0xd016b1963bce583998a8b2be795415211516856b2a49c4a7e2e49cecbd5d986b
"transfer"
	True
"transfer-tx_hash"
	0x716dc8e944018d02eecb0ce3b0a859b69d9628f74622c92d73eb1c3be40dfd7d_USDT
	multisig_0x6B49e6C79e604EA46E1DC8A936bFaD4Eb97D59E3_0xfefd77e047ba39845e2009f2833ab6177a93f2cd9cbb95965c809c16cb37680e_USDT
"similar_owner"
	False
"distributeEth"
	False
